In [ ]:
# imports
import numpy as np
import pandas as pd
import pickle
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from tqdm.auto import tqdm
from multiprocessing import Pool

In [ ]:
# connect to local elastic search host
HOST = 'http://localhost:9200/'
es = Elasticsearch(hosts=[HOST])
INDEX = 'info_clone'
NEWINDEX = 'info_clone2'
es.ping()

In [ ]:
t = es.indices.stats(index=INDEX)
SIZE = t['_all']['primaries']['docs']['count']

In [ ]:
s = Search(using=es, index=INDEX)
q = Q("wildcard", name='*')
query = s.query(q) #.extra(size=SIZE)
#response = query.execute()
#print(len(response))

In [ ]:
# Find files that contain no markdown or code
def check_empty(es, nid, name):
    mkdn = es.get(index='markdown', id=nid)
    mkdnlist = mkdn['_source']['markdown']
    code = es.get(index='code', id=nid)
    codelist = code['_source']['code']
    comments = es.get(index='comments2', id=nid)
    cmtlist = comments['_source']['comments']
    if mkdnlist == [] and codelist == [] and cmtlist == []:
        return True
    else:
        return False

In [ ]:
SLICES = 5                                                                      
empty_files = []
def dump_slice(slice_no):                                                       
    s = Search()                                                                
    s = s.extra(slice={"id": slice_no, "max": SLICES})                          
    for hit in s.scan():                                                          
        nid = hit.meta.id
        if check_empty(es, nid, name):
            empty_files.append(nid)

pool = Pool(SLICES)                                                             
pool.map(dump_slice, range(SLICES))

# empty_files = []
# for hit in s.scan():
#     nid = hit.meta.id
#     if check_empty(es, nid, name):
#         empty_files.append(nid)

In [ ]:
len(empty_files)

In [ ]:
with open('empty.pkl', 'wb') as f:
    pickle.dump(empty_files, f)

In [ ]:
with open('empty.pkl', 'rb') as f:
     empty_files = pickle.load(f)

In [ ]:
#es.delete(index=NEWINDEX, id=nid)